In [1]:
# Load dependencies
# Data processing
import pandas as pd

# Parallel processing
import swifter
import numpy as np
from multiprocessing import Pool

# Graphing
import matplotlib.pyplot as plt

# Misc
import json # JSON manipulation
import tldextract # Extract tlds
import idna # Punycode conversions
import re # Registry modifications
from datetime import datetime # Date calculations

# OUTPUT--------------

print("Successful.")

Successful.


In [2]:
# LOAD--------------------

# icann.csv
icann_df = pd.read_csv("data/icann.csv")

# Create tld-type dictionary
tld_type = pd.Series(icann_df.type.values, index=icann_df.tld).to_dict()

# registrar.csv
registrar_df = pd.read_csv("data/registrar.csv")

# Create registrar-id dictionary
registrar = pd.Series(registrar_df.registrar.values, index=registrar_df.id).to_dict()

# LOAD--------------------

# all_types_domain_balanced.json
df = pd.read_json("data/all_types_domains_balanced.json", orient="records", lines=True)

# Print # of null entries
before = len(df)
df = df[df["domain"].notnull()]
after = len(df)
print(f"Domains - nulls dropped: {(before - after)}")

# Print # of duplicates
before = len(df)
df = df.drop_duplicates(subset=["domain"])
after = len(df)
print(f"Domains - duplicates dropped: {(before - after)}")

# Print null entries
# df = df[df["domain"].isnull()]
# print(null_entries)
# df = df[df["domain"].notnull()]

# Print duplicates
# duplicates = df[df.duplicated(subset=["domain"], keep=False)]
# print(duplicates)
# df = df.drop_duplicates(subset=["domain"])

# LOAD--------------------

# all_types_domains_balanced_registered_domains_whois_parsed.json
# TODO: Uncomment duplicates after I figure out how to deal with them

whois_df = pd.read_json("data/all_types_domains_balanced_registered_domains_whois_parsed_correct_registrar_names_deduplicated_ids.json", orient="records", lines=True)

# Print # of null entries
before = len(whois_df)
whois_df = whois_df[whois_df["domain"].notnull()]
after = len(whois_df)
print(f'Whois - nulls dropped: {(before - after)}')

# Print # of duplicates
before = len(whois_df)
whois_df = whois_df.drop_duplicates(subset=["domain"])
after = len(whois_df)
print(f'Whois - duplicates dropped: {(before - after)}')

# Print # of malformed registrar ids
before = len(whois_df)
whois_df = whois_df[whois_df["registrar_id"].notnull()]
after = len(whois_df)
print(f'Whois - nulls dropped: {(before - after)}')

# Print null entries
# null_entries = whois_df[whois_df["domain"].isnull()]
# print(null_entries)
# whois_df = whois_df[whois_df["domain"].notnull()]

# Print duplicates
# duplicates = whois_df[whois_df.duplicated(subset=["domain"], keep=False)]
# print(duplicates)
# whois_df = whois_df.drop_duplicates(subset=["domain"])


# LOAD--------------------

# tld-list-history-2023-11-01.json
# Note: Don't need to drop the null entries or duplicates because there are no null entries and there are supposed to be duplicates 
tld_list_df = pd.read_csv("data/prices.csv")

# OUTPUT--------------

print("Successful.")

Domains - nulls dropped: 0
Domains - duplicates dropped: 99
Whois - nulls dropped: 2650
Whois - duplicates dropped: 198216
Whois - nulls dropped: 147
Successful.


In [3]:
# Extract TLDs and get the TLD type on dataframes

# APPLY--------------------

# all_types_domain_balanced.json
df["tld"] = df["domain"].swifter.apply(lambda x: tldextract.extract(x).suffix)
df["tld_type"] = df["tld"].swifter.apply(lambda x: tld_type.get(x, 'unknown'))

# APPLY--------------------

# all_types_domains_balanced_registered_domains_whois_parsed.json
# TLD Type
whois_df["tld"] = whois_df["domain"].swifter.apply(lambda x: tldextract.extract(x).suffix)
whois_df["tld_type"] = whois_df["tld"].swifter.apply(lambda x: tld_type.get(x, 'unknown'))

# Registrar
allowed_values = [9994, 9995, 9996, 9997, 9998, 9999, 10007, 10009, 4000001, 8888888]

whois_df['registrar_id'] = whois_df['registrar_id'].astype(int)

whois_df = whois_df[
    (whois_df['registrar_id'].isin(allowed_values)) |
    ((whois_df['registrar_id'] >= 57) & (whois_df['registrar_id'] <= 4325))
]

whois_df["registrar"] = whois_df["registrar_id"].swifter.apply(lambda x: registrar.get(x, 'unknown'))

# APPLY--------------------

# Convert to punycode
# tld-list-history-2023-11-01.json
tld_list_df['tld'] = tld_list_df['tld'].swifter.apply(lambda x: idna.encode(x).decode('utf-8') if any(ord(char) > 127 for char in x) else x)
tld_list_df = tld_list_df.sort_values(by='tld', ascending=True)

# OUTPUT--------------

print("Successful.")

Pandas Apply:   0%|          | 0/24429 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/24429 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/19686 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/19686 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/19603 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/4536493 [00:00<?, ?it/s]

Successful.


In [4]:
# Set up to add price data to whois_df from tld_list_df
# TODO: figure out how to determine whether to use renewal, register, or transfer price
# Note: I made copies since the not null dates still provides some useful info for malicious popularity

### FORMAT---------------------

# Drop null dates
before = len(whois_df)
whois_price = whois_df[whois_df["created"].notnull()].copy()
after = len(whois_price)
print(f"Whois - null dates dropped: {(before - after)}")

# No null tld_list dates
tld_list_price = tld_list_df[tld_list_df["date"].notnull()].copy()

# Format dates
whois_price["created"] = pd.to_datetime(whois_price["created"], format="%Y-%m-%d", errors='coerce')
tld_list_price["date"] = pd.to_datetime(tld_list_price["date"], format="%Y-%m-%d", errors='coerce')

# Drop misconfigured dates
before = len(whois_price)
whois_price = whois_price[whois_price["created"].notna()]
after = len(whois_price)
print(f'Whois - misconfigured dates dropped: {(before - after)}')

# No miscofigured tld_list dates
tld_list_price = tld_list_price[tld_list_price["date"].notna()]

### FILTER--------------------

# Filter for only register prices
tld_list_price = tld_list_price[tld_list_price["price-type"] == "register"]

# Group data by tld
tld_list_price = tld_list_price.groupby("tld")

### FUNCTIONS--------------------

# Function to find the price closest to the created date
def find_closest_price(row):
    try:
        # Retrieve the tld group
        tld_group = tld_list_price.get_group(row["tld"])
    except KeyError:
        # If there is no group, return none
        return pd.Series([None, None, None], index=["price", "price-date", "price-registrar"])
    
    # Compute days difference and find the closest date
    tld_group['days-difference'] = (row['created'] - tld_group['date']).dt.days
    closest_date = tld_group[tld_group['days-difference'] >= 0].nsmallest(1, 'days-difference')
    
    # Check if closest_date is empty
    if closest_date.empty:
        return pd.Series([None, None, None], index=["price", "price-date", "price-registrar"])

    # Concatenate the registrars (sadly not working rn)
    # registrars = ', '.join(set(closest_date['registrar']))
    return pd.Series([closest_date["price"].iloc[0], closest_date["date"].dt.date.iloc[0], closest_date["registrar"].iloc[0]], index=["price", "price-date", "price-registrar"])

# Function to split computation load among 10 cores
def apply_parallel(data, func):
    data_split = np.array_split(data, 10)
    with Pool(processes=10) as pool:
        data = pd.concat(pool.map(func, data_split))
    return data

# Function to run find_closest_price on each individualrow
def process_chunk(chunk):
    return chunk.apply(find_closest_price, axis=1)

# APPLY--------------------

# Apply function to the data frame
whois_price[["price", "price-date", "price-registrar"]] = apply_parallel(whois_price, process_chunk)

# FORMAT--------------------

# Make the none null dates human readable
whois_price["created"] = whois_price["created"].swifter.apply(lambda x: x.strftime("%Y-%m-%d") if pd.notnull(x) else x)
whois_price["price-date"] = whois_price["price-date"].swifter.apply(lambda x: x.strftime("%Y-%m-%d") if pd.notnull(x) else x)

# OUTPUT--------------

# Save whois_data to whois-with-prices.json
# whois_price.to_json("output/whois-with-prices.json", orient="records", lines=True)

print("Successful.")

Whois - null dates dropped: 699
Whois - misconfigured dates dropped: 112


Pandas Apply:   0%|          | 0/18792 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/18792 [00:00<?, ?it/s]

Successful.


In [6]:
# Graph time
# Convert date columns to datetime format
whois_price['created'] = pd.to_datetime(whois_price['created'], format="%Y-%m-%d", errors='coerce')
whois_price['updated'] = pd.to_datetime(whois_price['updated'], format="%Y-%m-%d", errors='coerce')
whois_price['expires'] = pd.to_datetime(whois_price['expires'], format="%Y-%m-%d", errors='coerce')
whois_price['price-date'] = pd.to_datetime(whois_price['price-date'], format="%Y-%m-%d", errors='coerce')

# Drop misconfigured dates
before = len(whois_price)
whois_price = whois_price.dropna(subset=['domain', 'created', 'expires'])
after = len(whois_price)
print(f'Whois - misconfigured dates dropped: {(before - after)}')

output_dir = './graphs/whois/'

# Distribution of Domain Prices
# Price range
price_range = (0, 10)
bin_width = 0.1
bins = int((price_range[1] - price_range[0]) / bin_width)

# Graphing/plotting
plt.figure(figsize=(10, 6))
plt.hist(whois_price['price'], bins=bins, range=price_range, color='skyblue', edgecolor='black')
plt.title('Distribution of Domain Prices')
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.xticks(range(0, 11, 1))
plt.savefig(f'{output_dir}distribution_of_domain_prices.pdf')
plt.close()
print("Created graph 1")

# Domains by Registrar (Top 20)
plt.figure(figsize=(14, 10))
whois_price['registrar'].value_counts().head(20).plot(kind='bar', color='lightgreen')
plt.title('Number of Domains by Registrar (Top 20)')
plt.xlabel('Registrar')
plt.ylabel('Number of Domains')
plt.xticks(rotation=90, fontsize=10)
plt.yticks(fontsize=10)

plt.tight_layout(pad=3.0)

plt.savefig(f'{output_dir}domains_by_registrar.pdf')
plt.close()
print("Created graph 2")

# Domains by TLD
plt.figure(figsize=(10, 8))
whois_price['tld'].value_counts().head(10).plot(kind='pie', autopct='%1.1f%%', startangle=90, colors=['gold', 'lightblue'])
plt.title('Distribution of Domains by TLD')
plt.ylabel('')

plt.tight_layout()

plt.savefig(f'{output_dir}domains_by_tld.pdf')
plt.close()
print("Created graph 3")

# Mean price by TLD
top_tlds = whois_price['tld'].value_counts().head(20).index

mean_prices = whois_price[whois_price['tld'].isin(top_tlds)].groupby('tld')['price'].mean().loc[top_tlds]

plt.figure(figsize=(10, 8))
mean_prices.plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Mean Price by TLD')
plt.xlabel('TLD')
plt.ylabel('Mean Price')
plt.xticks(rotation=45)
plt.tight_layout()

plt.savefig(f'{output_dir}mean_price_by_tld.pdf')
plt.close()
print("Created graph 4")

# Note that the graph below won't work since we are pulling the best price, not the actual price
# # Price by Registrar
# plt.figure(figsize=(10, 6))
# whois_price.groupby('registrar')['price'].mean().plot(kind='bar', color='lightcoral')
# plt.title('Average Price of Domains by Registrar')
# plt.xlabel('Registrar')
# plt.ylabel('Average Price')
# plt.xticks(rotation=45)
# plt.savefig(f'{output_dir}price_by_registrar.pdf')
# plt.close()
# print("Created graph 4")

print("Successful.")

Whois - misconfigured dates dropped: 0
Created graph 1
Created graph 2
Created graph 3
Created graph 4
Successful.


In [5]:
# List threat sources
sources = ['apt', 'phishtank', 'spamhaus', 'threatfox']

# Function to plot the graphs
def plot_tld(df, tld_type, source):
    # Filter by source
    if source is not None:
        filtered_df = df[df['source'] == source]
    else:
        filtered_df = df

    # Filter by TLD type
    if tld_type != 'overall':
        filtered_df = filtered_df[filtered_df['tld_type'] == tld_type]

    # Get the top 10
    top_10_df = filtered_df['tld'].value_counts(normalize=True).head(10)
    top_10_df['Other'] = 1 - top_10_df.sum()

    # Configure graph type
    plt.figure(figsize=(10,8))
    top_10_df.plot(kind='pie', autopct='%1.1f%%', startangle=90, colors=plt.cm.tab20.colors[:len(top_10_df)])

    # Configure graph file name and title
    if source is not None:
        title = f'Top 10 {tld_type.capitalize()} TLD Popularity for {source.capitalize()}'
        fname = f'./graphs/{source}_{tld_type}_popularity.pdf'
    else:
        title = f'Top 10 {tld_type.capitalize()} TLD Popularity'
        fname = f'./graphs/{source}_popularity.pdf'

    plt.title(title, fontsize=14)
    plt.ylabel('')
    plt.tight_layout()
    plt.savefig(fname, format='pdf')
    plt.clf()

# Create overall graphs
plot_tld(df, 'overall', None)
plot_tld(df, 'generic', None)
plot_tld(df, 'country-code', None)

# Create source graphs (apt, phishtank, spamhaus, threatfox)
for source in sources:
    plot_tld(df, 'overall', source)
    plot_tld(df, 'generic', source)
    plot_tld(df, 'country-code', source)

<Figure size 1000x800 with 0 Axes>

<Figure size 1000x800 with 0 Axes>

<Figure size 1000x800 with 0 Axes>

<Figure size 1000x800 with 0 Axes>

<Figure size 1000x800 with 0 Axes>

...

<Figure size 1000x800 with 0 Axes>

<Figure size 1000x800 with 0 Axes>

<Figure size 1000x800 with 0 Axes>

<Figure size 1000x800 with 0 Axes>

<Figure size 1000x800 with 0 Axes>